This is an example of a simple CNN developed, trained and utilized

AI was used to help generate the codebase

Note: Make sure that the tensorflow package is installed in your device.

In [89]:
# Lib imports
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
import numpy as np

In [90]:
# DATASET DIRECTORY CONFIGURATION
# Using garbage_classification dataset (paper vs plastic)
train_dir = "garbage_classification/train"  
test_dir = "garbage_classification/test"    

In [91]:
# IMAGE PARAMETERS
# Used to resize the input images, also will determine the input size of your input layer.
IMG_SIZE = (128, 128)
BATCH_SIZE = 32

In [ ]:
# DATA PREPROCESSING & AUGMENTATION
# Optional but recommended for image processing tasks, especially with limited data.
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    validation_split=0.2
)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='training'
)
val_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='validation'
)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle=False
)

Found 1532 images belonging to 2 classes.
Found 383 images belonging to 2 classes.
Found 1915 images belonging to 2 classes.


In [93]:
# IMPROVED CNN MODEL ARCHITECTURE WITH REGULARIZATION AND DROPOUT

# Some modifications are applied
initial_learning_rate = 0.001
# We are combining ExponentialDecay with Adam optimizer for better learning rate management
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate,
    decay_steps=10000,
    decay_rate=0.9,
    staircase=True
)

# Create the optimizer with the learning rate schedule
optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)

# Applied dropout layers and L2 regularization to reduce overfitting
# L2 regularization helps prevent overfitting by penalizing large weights
model = models.Sequential([
    layers.Input(shape=(IMG_SIZE[0], IMG_SIZE[1], 3)),
    layers.Conv2D(32, (3, 3), activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    layers.MaxPooling2D(2, 2),
    layers.Dropout(0.25),
    layers.Conv2D(64, (3, 3), activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    layers.MaxPooling2D(2, 2),
    layers.Dropout(0.25),
    layers.Conv2D(128, (3, 3), activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    layers.MaxPooling2D(2, 2),
    layers.Flatten(),
    layers.Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')
])

In [94]:
# Configure the model optimizers, loss function, and metrics
# model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy']) # old
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

In [95]:
# TRAINING THE CNN
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=val_generator
)

Epoch 1/10
48/48 ━━━━━━━━━━━━━━━━━━━━ 72s 1s/step - accuracy: 0.5521 - loss: 1.1863 - val_accuracy: 0.6762 - val_loss: 0.9034
Epoch 2/10
48/48 ━━━━━━━━━━━━━━━━━━━━ 24s 493ms/step - accuracy: 0.6315 - loss: 0.8354 - val_accuracy: 0.8146 - val_loss: 0.6376
Epoch 3/10
48/48 ━━━━━━━━━━━━━━━━━━━━ 27s 559ms/step - accuracy: 0.7116 - loss: 0.7183 - val_accuracy: 0.7311 - val_loss: 0.6216
Epoch 4/10
48/48 ━━━━━━━━━━━━━━━━━━━━ 24s 502ms/step - accuracy: 0.7809 - loss: 0.6036 - val_accuracy: 0.8120 - val_loss: 0.6664
Epoch 5/10
48/48 ━━━━━━━━━━━━━━━━━━━━ 25s 515ms/step - accuracy: 0.7315 - loss: 0.6255 - val_accuracy: 0.8407 - val_loss: 0.5104
Epoch 6/10
48/48 ━━━━━━━━━━━━━━━━━━━━ 24s 494ms/step - accuracy: 0.7813 - loss: 0.5385 - val_accuracy: 0.8486 - val_loss: 0.4399
Epoch 7/10
48/48 ━━━━━━━━━━━━━━━━━━━━ 25s 511ms/step - accuracy: 0.8051 - loss: 0.4729 - val_accuracy: 0.8460 - val_loss: 0.4135
Epoch 8/10
48/48 ━━━━━━━━━━━━━━━━━━━━ 25s 521ms/step - accuracy: 0.8262 - loss: 0.4386 - val_accurac

In [96]:
# EVALUATE THE MODEL
test_loss, test_acc = model.evaluate(test_generator, verbose=1)
print(f"\n{'='*50}")
print(f"IMPROVED MODEL - Test Accuracy: {test_acc:.4f} ({test_acc*100:.2f}%)")
print(f"Test Loss: {test_loss:.4f}")
print(f"{'='*50}")

60/60 ━━━━━━━━━━━━━━━━━━━━ 35s 595ms/step - accuracy: 0.9140 - loss: 0.2828

IMPROVED MODEL - Test Accuracy: 0.7232 (72.32%)
Test Loss: 0.6300


In [97]:
# SAVE THE IMPROVED MODEL
model.save('muffin_vs_chihuahua_cnn_improved.keras')
print("Improved model saved as 'muffin_vs_chihuahua_cnn_improved.keras'")

Improved model saved as 'muffin_vs_chihuahua_cnn_improved.keras'


In [98]:
# INFERENCE SCRIPT FOR IMPROVED MODEL
from tensorflow.keras.preprocessing import image

# Load model once for efficiency
_loaded_model = None

def predict_image(img_path, model_path='muffin_vs_chihuahua_cnn_improved.keras', img_size=IMG_SIZE):
    global _loaded_model
    if _loaded_model is None:
        _loaded_model = tf.keras.models.load_model(model_path)
    img = image.load_img(img_path, target_size=img_size)
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    pred = _loaded_model.predict(img_array, verbose=0)[0,0]
    label = "Plastic" if pred >= 0.5 else "Paper"  # Class 0=Paper, Class 1=Plastic (alphabetical order)
    confidence = pred if pred >= 0.5 else (1 - pred)
    print(f"Image: {img_path}")
    print(f"Prediction: {label}")
    print(f"Confidence: {confidence:.4f} ({confidence*100:.2f}%)")
    print(f"Raw prediction value: {pred:.4f}")
    print("-" * 50)
    return label, pred, confidence


In [99]:
predict_image("run_1/run_1.jpg")
predict_image("run_1/run_2.jpg")

Image: run_1/run_1.jpg
Prediction: Paper
Confidence: 0.9900 (99.00%)
Raw prediction value: 0.0100
--------------------------------------------------
Image: run_1/run_2.jpg
Prediction: Paper
Confidence: 0.9936 (99.36%)
Raw prediction value: 0.0064
--------------------------------------------------


('Paper', np.float32(0.0063803317), np.float32(0.9936197))

In [100]:
# IMPROVED CNN MODEL WITH REGULARIZATION AND DROPOUT


# Learning rate schedule
initial_learning_rate = 0.001
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate,
    decay_steps=10000,
    decay_rate=0.9,
    staircase=True
)

# Create the optimizer with the learning rate schedule
optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)

# Improved model with dropout and L2 regularization
# L2 regularization (lambda=0.001) penalizes large weights to prevent overfitting
# Dropout layers randomly deactivate neurons during training to improve generalization
model_improved = models.Sequential([
    layers.Input(shape=(IMG_SIZE[0], IMG_SIZE[1], 3)),
    layers.Conv2D(32, (3, 3), activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    layers.MaxPooling2D(2, 2),
    layers.Dropout(0.25),
    layers.Conv2D(64, (3, 3), activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    layers.MaxPooling2D(2, 2),
    layers.Dropout(0.25),
    layers.Conv2D(128, (3, 3), activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    layers.MaxPooling2D(2, 2),
    layers.Flatten(),
    layers.Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')
])

# Compile the improved model
model_improved.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

print("Improved model architecture created with:")
print("- L2 regularization (lambda=0.001) on all Conv2D and Dense layers")
print("- Dropout layers: 0.25 after conv layers, 0.5 after dense layer")


Improved model architecture created with:
- L2 regularization (lambda=0.001) on all Conv2D and Dense layers
- Dropout layers: 0.25 after conv layers, 0.5 after dense layer


In [101]:
# TRAINING THE IMPROVED CNN MODEL
print("Training improved model with regularization and dropout...")
history_improved = model_improved.fit(
    train_generator,
    epochs=10,
    validation_data=val_generator,
    verbose=1
)


Training improved model with regularization and dropout...
Epoch 1/10
48/48 ━━━━━━━━━━━━━━━━━━━━ 32s 580ms/step - accuracy: 0.4975 - loss: 1.3240 - val_accuracy: 0.5483 - val_loss: 0.9144
Epoch 2/10
48/48 ━━━━━━━━━━━━━━━━━━━━ 27s 562ms/step - accuracy: 0.5604 - loss: 0.8804 - val_accuracy: 0.5979 - val_loss: 0.8346
Epoch 3/10
48/48 ━━━━━━━━━━━━━━━━━━━━ 25s 520ms/step - accuracy: 0.6863 - loss: 0.7416 - val_accuracy: 0.7885 - val_loss: 0.6410
Epoch 4/10
48/48 ━━━━━━━━━━━━━━━━━━━━ 25s 517ms/step - accuracy: 0.7409 - loss: 0.6384 - val_accuracy: 0.8303 - val_loss: 0.5018
Epoch 5/10
48/48 ━━━━━━━━━━━━━━━━━━━━ 25s 527ms/step - accuracy: 0.7518 - loss: 0.5903 - val_accuracy: 0.8146 - val_loss: 0.4641
Epoch 6/10
48/48 ━━━━━━━━━━━━━━━━━━━━ 25s 524ms/step - accuracy: 0.8072 - loss: 0.5127 - val_accuracy: 0.7781 - val_loss: 0.5287
Epoch 7/10
48/48 ━━━━━━━━━━━━━━━━━━━━ 28s 590ms/step - accuracy: 0.8082 - loss: 0.4698 - val_accuracy: 0.8251 - val_loss: 0.4011
Epoch 8/10
48/48 ━━━━━━━━━━━━━━━━━━━━ 

In [102]:
# EVALUATE THE IMPROVED MODEL
print("EVALUATING IMPROVED MODEL")
test_loss_improved, test_acc_improved = model_improved.evaluate(test_generator, verbose=1)
print(f"IMPROVED MODEL RESULTS:")
print(f"Test Accuracy: {test_acc_improved:.4f} ({test_acc_improved*100:.2f}%)")
print(f"Test Loss: {test_loss_improved:.4f}")

#2a: Accuracy
print("2a) ACCURACY: ")
print(f"The improved model achieves a test accuracy of {test_acc_improved:.4f} ({test_acc_improved*100:.2f}%)")

EVALUATING IMPROVED MODEL


60/60 ━━━━━━━━━━━━━━━━━━━━ 9s 153ms/step - accuracy: 0.9277 - loss: 0.2681
IMPROVED MODEL RESULTS:
Test Accuracy: 0.7864 (78.64%)
Test Loss: 0.5251
2a) ACCURACY: 
The improved model achieves a test accuracy of 0.7864 (78.64%)


In [103]:
# SAVE THE IMPROVED MODEL
model_improved.save('exercise_6_custom_grecia.h5')
print("Improved model saved as 'exercise_6_custom_lastname.grecia.h5'")


Improved model saved as 'exercise_6_custom_lastname.grecia.h5'


In [104]:
# PREDICTIONS ON RUN 1 AND RUN 2 USING IMPROVED MODEL
from tensorflow.keras.preprocessing import image

# Load the improved model
_loaded_model_improved = None

def predict_image_improved(img_path, model_path='exercise_6_trained_model_improved.h5', img_size=IMG_SIZE):
    global _loaded_model_improved
    if _loaded_model_improved is None:
        _loaded_model_improved = tf.keras.models.load_model(model_path)
    img = image.load_img(img_path, target_size=img_size)
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    pred = _loaded_model_improved.predict(img_array, verbose=0)[0,0]
    label = "Plastic" if pred >= 0.5 else "Paper" 
    confidence = pred if pred >= 0.5 else (1 - pred)
    return label, pred, confidence

print("IMPROVED MODEL PREDICTIONS ON RUN 1 AND RUN 2")

print("\nRun 1:")
label1, pred1, conf1 = predict_image_improved("run_1/run_1.jpg")
print(f"  Image: run_1/run_1.jpg")
print(f"  Prediction: {label1}")
print(f"  Confidence: {conf1:.4f} ({conf1*100:.2f}%)")
print(f"  Raw prediction value: {pred1:.4f}")

print("\nRun 2:")
label2, pred2, conf2 = predict_image_improved("run_1/run_2.jpg")
print(f"  Image: run_1/run_2.jpg")
print(f"  Prediction: {label2}")
print(f"  Confidence: {conf2:.4f} ({conf2*100:.2f}%)")
print(f"  Raw prediction value: {pred2:.4f}")

print("2b): RUN 1 AND 2 PREDICTIONS")
print(f"Run 1 Prediction: {label1}")
print(f"Run 1 Confidence: {conf1:.4f} ({conf1*100:.2f}%)")
print(f"\nRun 2 Prediction: {label2}")
print(f"Run 2 Confidence: {conf2:.4f} ({conf2*100:.2f}%)")


IMPROVED MODEL PREDICTIONS ON RUN 1 AND RUN 2

Run 1:


  Image: run_1/run_1.jpg
  Prediction: Plastic
  Confidence: 0.8273 (82.73%)
  Raw prediction value: 0.8273

Run 2:
  Image: run_1/run_2.jpg
  Prediction: Plastic
  Confidence: 0.5565 (55.65%)
  Raw prediction value: 0.5565
2b): RUN 1 AND 2 PREDICTIONS
Run 1 Prediction: Plastic
Run 1 Confidence: 0.8273 (82.73%)

Run 2 Prediction: Plastic
Run 2 Confidence: 0.5565 (55.65%)
